In [2]:
import psycopg2
# import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [3]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [4]:
soil_df = pd.read_sql("select sample_code, batch_date, analysis_name, chemical_name, result, unit_name from SampleResults where group_name='Soil Analysis'",con=conn_lims)

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
soil_df.to_csv("input/soil_samples.csv")

In [8]:
soil_df = pd.read_csv("input/soil_samples.csv")

In [6]:
soil_df

,sample_code,batch_date,crop_name,chemical_name,result,unit_name
0,CF045SA0784,2012-07-09,Tomatoes,Iron,165,ppm
1,CF045SA0784,2012-07-09,Tomatoes,C.E.C,16.6,meq/100g
2,CF045SA0784,2012-07-09,Tomatoes,Aluminium,481,ppm
3,CF045SA0784,2012-07-09,Tomatoes,EC (Salts),61.0,uS/cm
4,CF045SA0784,2012-07-09,Tomatoes,Organic Matter,2.36,%
...,...,...,...,...,...,...
4038895,CI118SA1115,2024-04-10,No crop,EDTA Zinc,1.51,ppm
4038896,CI118SA1115,2024-04-10,No crop,EDTA Iron,134,ppm
4038897,CI118SA1115,2024-04-10,No crop,EDTA manganese,84.0,ppm
4038898,CI118SA1115,2024-04-10,No crop,Total Nitrogen,0.10,%


In [6]:
soil_df['analysis_name'].value_counts()

Complete Soil Analysis with Recommendations    1271546
AFSIS Std Wet Chemistry Soil Analysis           198825
Complete Soil Analysis                          191014
Complete Soil Analysis (M3)                     180924
Basic Soil Analysis with Recommendations        143443
                                                ...   
Listeria spp                                         2
Salmonella                                           2
Starter Soil Scan (IR)                               1
Gold Analysis                                        1
Salmonella spp                                       1
Name: analysis_name, Length: 906, dtype: int64

In [7]:
soil_df = soil_df.dropna(subset='result')

In [8]:
soil_df = soil_df.loc[~(soil_df['result'] == "ND") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Detected") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Clay") ]
soil_df = soil_df.loc[~(soil_df['result'] == "HClay") ]
soil_df = soil_df.loc[~(soil_df['result'] == "CLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "Loam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SCLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SLoam") ]
soil_df = soil_df.loc[~(soil_df['result'] == "LSand") ]
soil_df = soil_df.loc[~(soil_df['result'] == "SClay") ]

In [9]:
soil_df['result'] = [ (i.replace("<","").replace(">","").strip()) for i in soil_df['result']]

In [10]:
soil_df = soil_df.loc[soil_df['result'] != ""]

In [11]:
soil_df

,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",Iron,165,ppm
1,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",C.E.C,16.6,meq/100g
2,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",Aluminium,481,ppm
3,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",EC (Salts),61.0,uS/cm
4,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",Organic Matter,2.36,%
...,...,...,...,...,...,...
4038895,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",EDTA Zinc,1.51,ppm
4038896,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",EDTA Iron,134,ppm
4038897,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",EDTA manganese,84.0,ppm
4038898,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",Total Nitrogen,0.10,%


In [12]:
soil_df['unit_name'] = [ str(i).strip() for i in soil_df['unit_name'].values ]

In [13]:
soil_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in soil_df['chemical_name'].values ]

In [14]:
soil_df = soil_df.loc[~(soil_df['result'].astype("float32") < 0)]

In [15]:
soil_df = soil_df.loc[~(soil_df['unit_name'] == "nan")]

In [16]:
soil_df

,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",iron,165,ppm
1,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",cec,16.6,meq/100g
2,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",aluminium,481,ppm
3,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",ec_salts,61.0,uS/cm
4,CF045SA0784,2012-07-09,"% Organic Matter (OM),Complete Soil Analysis",organic_matter,2.36,%
...,...,...,...,...,...,...
4038895,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",edta_zinc,1.51,ppm
4038896,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",edta_iron,134,ppm
4038897,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",edta_manganese,84.0,ppm
4038898,CI118SA1115,2024-04-10,"Boron Analysis,Organic Carbon & Total N (Wetch...",total_nitrogen,0.10,%


In [17]:
soil_df.to_csv("input/soil_analysis_cleaned.csv")